In [ ]:
%%cmd
python -m pip install flatten_json

In [ ]:
import pandas as pd
import json
from collections import defaultdict
import numpy as np
import argparse
import sys
from flatten_json import flatten
import matplotlib.pyplot as plt

In [ ]:
def aggregate_loops_passes(json):
    results_per_frame = []
    num_loops = len(json)
    for loop_results in json:
        for frame_index, frame_results in enumerate(loop_results["per_frame_results"]):
            #pprint.pprint(frame_results)
            if frame_index >= len(results_per_frame):
                results_per_frame.append(defaultdict(int))
            for command_buffer_timings in frame_results[
                "command_buffer_timings"
            ].values():
                for scope_name, scope_timings in command_buffer_timings[
                    "scope_timings"
                ].items():
                    for scope_timing in scope_timings:
                        results_per_frame[frame_index][scope_name] += (
                            scope_timing["end"] - scope_timing["start"]
                        ) / num_loops / 1_000_000 # in ms
            for metric_name, metric in frame_results["metrics"].items():
                # TODO: Flatten this in rust to fan_speed_rpm
                if metric_name == "fan_speed":
                    value = metric["Percent"] if "Percent" in metric else metric["Rpm"]
                    results_per_frame[frame_index]["fan_speed_rpm"] += (
                        value / num_loops
                    )
                # Filter out unavailable data and the timestamp
                elif metric is not None and metric_name != "timestamp":
                    results_per_frame[frame_index][metric_name] += metric / num_loops
    # TODO: Aggregate CPU timings
    return pd.DataFrame([flatten(x) for x in results_per_frame])


def metric_names():
    return [
        "edge_temperature_in_c",
        "hotspot_temperature_in_c",
        "usage_percentage",
        "fan_speed_rpm",
        "clock_speed_in_mhz",
        "vram_clock_speed_in_mhz",
        "board_power_usage_in_w",
        "voltage_in_mv",
        "vram_usage_in_mb",
    ]

In [ ]:
files = [
    #'../../a_path_tracing_workgraph.csv',
    #'../../b_path_tracing_workgraph.csv',
    '../../a_ray_tracing_inline.csv',
    '../../b_ray_tracing_inline.csv',
    '../../a_ray_tracing_pipeline.csv',
    '../../b_ray_tracing_pipeline.csv',
    '../../b_loop.csv'
]

scores = {}

for file in files:
    scores[file] = pd.read_csv(file)
scores = pd.concat(scores).groupby(level=0, sort=False).mean().T.drop('Loop', errors="ignore")
scores

In [ ]:
scores.plot(kind="bar", figsize=(20,5), colormap='Dark2', grid=True, rot=0)

In [ ]:
files = [
    '../../a_ray_tracing_inline_deep.json',
    '../../b_ray_tracing_inline_deep.json',
    #'../../a_ray_tracing_pipeline_deep.json',
    #'../../b_ray_tracing_pipeline_deep.json',
    #'../../a_path_tracing_workgraph_deep.json',
    #'../../b_path_tracing_workgraph_deep.json',
]

results = {}

for path in files:
    with open(path, "r") as json_file:
        json_data = aggregate_loops_passes(json.load(json_file))
    results[path] = json_data
result = pd.concat(results)
result

In [ ]:
result.columns.tolist()

In [ ]:
relevant_metrics = [
    'apply_wind_to_vertices',
    'usage_percentage',
    'vbuffer-mesh-parts-prefix-sum'
]
metrics = result[metric_names()]

In [ ]:
filtered_metrics = metrics.stack().unstack(level=0).reset_index()

In [ ]:
filtered_metrics.groupby('level_1').plot(x='level_0', xlabel='frame', figsize=(20,5), colormap='Dark2', grid=True, legend=True)